The purpose of this notebook is to train the model with arcface and sphereface loss function and test with our databases.

# pip install requirements

Remember to click on "Restart Runtime" before go on

In [ ]:
# CosPlace requirements
!pip3 install "faiss_cpu>=1.7.1"
!pip3 install "numpy>=1.21.2"
!pip3 install "Pillow>=9.0.1"
!pip3 install "scikit_learn>=1.0.2"
!pip3 install "torch>=1.8.2"
!pip3 install "torchvision>=0.9.2"
!pip3 install "tqdm>=4.62.3"
!pip3 install "utm>=0.7.0"

import torch
#use GPU if available 
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #'cpu' # 'cuda' or 'cpu'
print(DEVICE)

: 

# Download Datasets and previous data

Downloading with gdown doesn't work properly.

Prefer always to use drive / mount

In [ ]:
import os
import gdown
from google.colab import drive

def download(id, output=None, quiet=True):
  gdown.download(
    f"https://drive.google.com/uc?export=download&confirm=pbef&id={id}",
    output=output,
    quiet=quiet
  )


use_mount = True
resume_model = False

if use_mount:
  drive.mount('/content/drive')

# TOKYO-XS DATASET
if not os.path.isdir("/content/tokyo_xs"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/tokyo-xs.zip"
  else:
    id = "1fBCnap5BRh36474cVkjvjlC-yUTEb1n3"
    download(id, quiet=False)                           # download from our gdrive
    !jar xvf "/content/tokyo-xs.zip"                    # unzip
    !rm -r "/content/tokyo-xs.zip"                      # remove .zip file

#TOKYO NIGHT DATASET
if not os.path.isdir("/content/tokyo_night"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/tokyo-night.zip"
  else:
    id = "1EZJY2r5565-iVk2oEbTns0xc4F_em4Y4"
    download(id, quiet=False)                           # download from our gdrive
    !jar xvf "/content/tokyo-night.zip"                    # unzip
    !rm -r "/content/tokyo-night.zip"

# SAN FRANCISCO - XS DATASET
if not os.path.isdir("/content/small"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/sf-xs.zip"
  else:
    id = "1brIxBJmOgvuzFbI57f5LxnMxjccUu993"
    download(id, quiet=False)                           # download
    !jar xvf "/content/sf-xs.zip"                       # unzip
    !rm -r "/content/sf-xs.zip"                         # remove .zip file

# resumed model
if resume_model and not os.path.isfile("/content/saved_models/arcface_model.pth"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/saved_models.zip"

if resume_model and not os.path.isfile("/content/saved_models/sphereface_model.pth"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/saved_models.zip"

if resume_model and not os.path.isfile("/content/saved_models/cosface_model.pth"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/saved_models.zip"

# Download Code

Clone of original repo of CosPlace and our code

In [ ]:
# download code of CosPlace
!git clone "https://github.com/gmberton/CosPlace" 
#!rm -r "/content/CosPlace"

# download our code
!git clone --single-branch --branch "sam" "https://github.com/gab-palmeri/aml-geolocalization.git"
!mv "/content/aml-geolocalization/" "/content/Team"
#!rm -r "/content/aml-geolocalization"



# Import Code


In [ ]:
import os
import sys
import torch
import logging
import multiprocessing
import numpy as np
import torchvision.transforms as T
from tqdm import tqdm
from datetime import datetime

sys.path.append("/content/CosPlace/")
sys.path.append("/content/Team/")
import CosPlace
from CosPlace import *

torch.backends.cudnn.benchmark = True  # Provides a speedup

This class let us to access to dictionary keys like `dict.key` instead of `dict["key"]`

In [ ]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

# Parameters

Original code provides two main executable files: `train.py` and `eval.py`. We want to reproduce the same behaviour of these two files so these are the parameters passed via terminal. They will be put in a dict called `args` to simply reuse code inside ex files

### Save dir for ArcFace

In [ ]:
saveDirLocal = "arcface"

### Save dir for SphereFace

In [ ]:
saveDirLocal = "sphereface"

### Save dir for CosFace

In [ ]:
saveDirLocal = "cosface"

In [ ]:
# CosPlace Groups parameters
COS_M = 10
ALPHA = 30
COS_N = 5
COS_L = 2
GROUPS_NUM = 1
MIN_IMAGES_PER_CLASS = 10
# Model parameters
BACKBONE = "resnet18"   # ["vgg16", "resnet18", "resnet50", "resnet101", "resnet152"]
FC_OUTPUT_DIM = 512     # Output dimension of final fully connected layer
# Training parameters
USE_AMP_16 = "store_true"       # use Automatic Mixed Precision
AUGMENTATION_DEVICE = "cuda"    # ["cuda", "cpu"]
BATCH_SIZE = 32
EPOCHS_NUM = 3
ITERATIONS_PER_EPOCH = 10_000
LR = 0.00001                    # Learning rate  
CLASSIFIERS_LR = 0.01
# Data augmentation
BRIGHTNESS = 0.7
CONTRAST = 0.7
SATURATION = 0.7
HUE = 0.5
RANDOM_RESIZED_CROP = 0.5
# Validation / test parameters
INFER_BATCH_SIZE = 16           # Batch size for inference (validating and testing)
POSITIVE_DIST_THRESHOLD = 25    # distance in meters for a prediction to be considered a positive
# Resume parameters
RESUME_TRAIN = None     # path to checkpoint to resume, e.g. logs/.../last_checkpoint.pth
RESUME_MODEL = None     # Path to model to resume training from
# Other parameters
DEVICE = "cuda"                     # ["cuda", "cpu"]
SEED = 0
NUM_WORKERS = 8
DATASET_FOLDER = "/content/small"   # path of the folder with train/val sets
SAVEDIR = saveDirLocal


if not os.path.exists(DATASET_FOLDER):
    raise FileNotFoundError(f"Dataset folder {DATASET_FOLDER} not found")

train_set_folder = os.path.join(DATASET_FOLDER, "train")

if not os.path.exists(train_set_folder):
    raise FileNotFoundError(f"Train set folder {train_set_folder} not found")

val_set_folder = os.path.join(DATASET_FOLDER, "val")

if not os.path.exists(val_set_folder):
    raise FileNotFoundError(f"Validation set folder {val_set_folder} not found")


# dictionary for the parameters
args = {
    'M': COS_M, 'alpha': ALPHA, 'N': COS_N, 'L': COS_L, 'groups_num': GROUPS_NUM,
    'min_images_per_class': MIN_IMAGES_PER_CLASS, 'backbone': BACKBONE,
    'fc_output_dim': FC_OUTPUT_DIM, 'use_amp_16': USE_AMP_16,
    'augmentation_device': AUGMENTATION_DEVICE, 'batch_size': BATCH_SIZE,
    'epochs_num': EPOCHS_NUM, 'iterations_per_epoch': ITERATIONS_PER_EPOCH,
    'lr': LR, 'classifiers_lr': CLASSIFIERS_LR, 'brightness': BRIGHTNESS,
    'contrast': CONTRAST, 'hue': HUE, 'saturation': SATURATION,
    'random_resized_crop': RANDOM_RESIZED_CROP, 'infer_batch_size': INFER_BATCH_SIZE,
    'positive_dist_threshold': POSITIVE_DIST_THRESHOLD, 'resume_train': RESUME_TRAIN,
    'resume_model': RESUME_MODEL, 'device': DEVICE, 'seed': SEED,
    'num_workers': NUM_WORKERS, 'dataset_folder': DATASET_FOLDER, 'save_dir': SAVEDIR,
    'val_set_folder': val_set_folder, 'train_set_folder': train_set_folder,
}

# this helps to reuse the code from the original CosPlace
args = dotdict(args)


# Setup logging

In [ ]:
from CosPlace import commons

start_time = datetime.now()
output_folder = f"logs/{args.save_dir}/{start_time.strftime('%Y-%m-%d_%H-%M-%S')}"
commons.make_deterministic(args.seed)
commons.setup_logging(output_folder, console=None)
logging.info(" ".join(sys.argv))
logging.info(f"Arguments: {args}")
logging.info(f"The outputs are being saved in {output_folder}")

# PAY ATTENTION!
If you want to just test an already provided model, you should skip the training part and go to [Test section](#scrollTo=Y5jJf7v5Ox91)

# Training



## Model

In [ ]:
from CosPlace import model
from model import network

model = network.GeoLocalizationNet(args.backbone, args.fc_output_dim)

logging.info(f"There are {torch.cuda.device_count()} GPUs and {multiprocessing.cpu_count()} CPUs.")

if args.resume_model is not None:
    logging.debug(f"Loading model from {args.resume_model}")
    model_state_dict = torch.load(args.resume_model)
    model.load_state_dict(model_state_dict)

model = model.to(args.device).train()

## Optimizer

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
model_optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

## Datasets

#### ArcFace

In [ ]:
from CosPlace import datasets
from datasets.train_dataset import TrainDataset
from datasets.test_dataset import TestDataset
from Team.loss.arcface import ArcFace
from Team.loss.cosface import CosFace

groups = [TrainDataset(args, args.train_set_folder, M=args.M, alpha=args.alpha, N=args.N, L=args.L,
                       current_group=n, min_images_per_class=args.min_images_per_class) for n in range(args.groups_num)]

# apply arcface loss to each group
# Each group has its own classifier, which depends on the number of classes in the group
classifiers = [ArcFace(args.fc_output_dim, len(group)) for group in groups]
classifiers_optimizers = [torch.optim.Adam(classifier.parameters(), lr=args.classifiers_lr) for classifier in classifiers]

logging.info(f"Using {len(groups)} groups")
logging.info(f"The {len(groups)} groups have respectively the following number of classes {[len(g) for g in groups]}")
logging.info(f"The {len(groups)} groups have respectively the following number of images {[g.get_images_num() for g in groups]}")

val_ds = TestDataset(args.val_set_folder, positive_dist_threshold=args.positive_dist_threshold)
logging.info(f"Validation set: {val_ds}")

#### SphereFace

In [ ]:
from CosPlace import datasets
from datasets.train_dataset import TrainDataset
from datasets.test_dataset import TestDataset
from Team.loss.sphereface import SphereFace

groups = [TrainDataset(args, args.train_set_folder, M=args.M, alpha=args.alpha, N=args.N, L=args.L,
                       current_group=n, min_images_per_class=args.min_images_per_class) for n in range(args.groups_num)]

# apply sphereface loss to each group
# Each group has its own classifier, which depends on the number of classes in the group
classifiers = [SphereFace(args.fc_output_dim, len(group)) for group in groups]
classifiers_optimizers = [torch.optim.Adam(classifier.parameters(), lr=args.classifiers_lr) for classifier in classifiers]

logging.info(f"Using {len(groups)} groups")
logging.info(f"The {len(groups)} groups have respectively the following number of classes {[len(g) for g in groups]}")
logging.info(f"The {len(groups)} groups have respectively the following number of images {[g.get_images_num() for g in groups]}")

val_ds = TestDataset(args.val_set_folder, positive_dist_threshold=args.positive_dist_threshold)
logging.info(f"Validation set: {val_ds}")

### CosFace

In [ ]:
from CosPlace import datasets
from datasets.train_dataset import TrainDataset
from datasets.test_dataset import TestDataset
from Team.loss.cosface import CosFace

groups = [TrainDataset(args, args.train_set_folder, M=args.M, alpha=args.alpha, N=args.N, L=args.L,
                       current_group=n, min_images_per_class=args.min_images_per_class) for n in range(args.groups_num)]

# apply cosface loss to each group
# Each group has its own classifier, which depends on the number of classes in the group
classifiers = [CosFace(args.fc_output_dim, len(group)) for group in groups]
classifiers_optimizers = [torch.optim.Adam(classifier.parameters(), lr=args.classifiers_lr) for classifier in classifiers]

logging.info(f"Using {len(groups)} groups")
logging.info(f"The {len(groups)} groups have respectively the following number of classes {[len(g) for g in groups]}")
logging.info(f"The {len(groups)} groups have respectively the following number of images {[g.get_images_num() for g in groups]}")

val_ds = TestDataset(args.val_set_folder, positive_dist_threshold=args.positive_dist_threshold)
logging.info(f"Validation set: {val_ds}")

## Resume train

In [ ]:
from CosPlace import util

if args.resume_train:
    model, model_optimizer, classifiers, classifiers_optimizers, best_val_recall1, start_epoch_num = \
        util.resume_train(args, output_folder, model, model_optimizer, classifiers, classifiers_optimizers)
    model = model.to(args.device)
    epoch_num = start_epoch_num - 1
    logging.info(f"Resuming from epoch {start_epoch_num} with best R@1 {best_val_recall1:.1f} from checkpoint {args.resume_train}")
else:
    best_val_recall1 = start_epoch_num = 0

## Train and Evaluation Loop

In [ ]:
from CosPlace import augmentations, test

logging.info("Start training ...")
logging.info(f"There are {len(groups[0])} classes for the first group, " +
             f"each epoch has {args.iterations_per_epoch} iterations " +
             f"with batch_size {args.batch_size}, therefore the model sees each class (on average) " +
             f"{args.iterations_per_epoch * args.batch_size / len(groups[0]):.1f} times per epoch")


if args.augmentation_device == "cuda":
    gpu_augmentation = T.Compose([
            augmentations.DeviceAgnosticColorJitter(brightness=args.brightness,
                                                    contrast=args.contrast,
                                                    saturation=args.saturation,
                                                    hue=args.hue),
            augmentations.DeviceAgnosticRandomResizedCrop([512, 512],
                                                          scale=[1-args.random_resized_crop, 1]),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

if args.use_amp16:
    scaler = torch.cuda.amp.GradScaler()

for epoch_num in range(start_epoch_num, args.epochs_num):
    
    #### Train
    epoch_start_time = datetime.now()
    # Select classifier and dataloader according to epoch
    current_group_num = epoch_num % args.groups_num
    classifiers[current_group_num] = classifiers[current_group_num].to(args.device)
    util.move_to_device(classifiers_optimizers[current_group_num], args.device)
    
    dataloader = commons.InfiniteDataLoader(groups[current_group_num], num_workers=args.num_workers,
                                            batch_size=args.batch_size, shuffle=True,
                                            pin_memory=(args.device == "cuda"), drop_last=True)
    
    dataloader_iterator = iter(dataloader)
    model = model.train()
    
    epoch_losses = np.zeros((0, 1), dtype=np.float32)
    for iteration in tqdm(range(args.iterations_per_epoch), ncols=100):
        images, targets, _ = next(dataloader_iterator)
        images, targets = images.to(args.device), targets.to(args.device)
        
        if args.augmentation_device == "cuda":
            images = gpu_augmentation(images)
        
        model_optimizer.zero_grad()
        classifiers_optimizers[current_group_num].zero_grad()
        
        if not args.use_amp16:
            descriptors = model(images)
            output = classifiers[current_group_num](descriptors, targets)
            loss = criterion(output, targets)
            loss.backward()
            epoch_losses = np.append(epoch_losses, loss.item())
            del loss, output, images
            model_optimizer.step()
            classifiers_optimizers[current_group_num].step()
        else:  # Use AMP 16
            with torch.cuda.amp.autocast():
                descriptors = model(images)
                output = classifiers[current_group_num](descriptors, targets)
                loss = criterion(output, targets)
            scaler.scale(loss).backward()
            epoch_losses = np.append(epoch_losses, loss.item())
            del loss, output, images
            scaler.step(model_optimizer)
            scaler.step(classifiers_optimizers[current_group_num])
            scaler.update()
    
    classifiers[current_group_num] = classifiers[current_group_num].cpu()
    util.move_to_device(classifiers_optimizers[current_group_num], "cpu")
    
    logging.debug(f"Epoch {epoch_num:02d} in {str(datetime.now() - epoch_start_time)[:-7]}, "
                  f"loss = {epoch_losses.mean():.4f}")
    
    #### Evaluation
    recalls, recalls_str = test.test(args, val_ds, model)
    logging.info(f"Epoch {epoch_num:02d} in {str(datetime.now() - epoch_start_time)[:-7]}, {val_ds}: {recalls_str[:20]}")
    is_best = recalls[0] > best_val_recall1
    best_val_recall1 = max(recalls[0], best_val_recall1)
    # Save checkpoint, which contains all training parameters
    util.save_checkpoint({
        "epoch_num": epoch_num + 1,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": model_optimizer.state_dict(),
        "classifiers_state_dict": [c.state_dict() for c in classifiers],
        "optimizers_state_dict": [c.state_dict() for c in classifiers_optimizers],
        "best_val_recall1": best_val_recall1
    }, is_best, output_folder)


logging.info(f"Trained for {epoch_num+1:02d} epochs, in total in {str(datetime.now() - start_time)[:-7]}")

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
INFO:root:Start training ...
INFO:root:There are 5965 classes for the first group, each epoch has 10000 iterations with batch_size 32, therefore the model sees each class (on average) 53.6 times per epoch
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|███████████████████████████████████████████████████████| 10000/10000 [1:14:13<00:00,  2.25it/s]
DEBUG:root:Epoch 00 in 1:14:14, loss = 7.9503
DEBUG:root:Extracting database descriptors for evaluation/te

# Test

## Import model

In [ ]:

from CosPlace import model, test, datasets
from model import network
from datasets.test_dataset import TestDataset

#### Model
model = network.GeoLocalizationNet(args.backbone, args.fc_output_dim)

logging.info(f"There are {torch.cuda.device_count()} GPUs and {multiprocessing.cpu_count()} CPUs.")

# set the path to the model you want to resume
# the following line will take the model trained here
# resume_model = f"/content/logs/default/{start_time.strftime('%Y-%m-%d_%H-%M-%S')}/best_model.pth"
if args.resume_model is not None:
  if os.path.exists(args.resume_model):
    resume_model = args.resume_model
if os.path.exists(resume_model):
  logging.info(f"Loading model from {resume_model}")
  model_state_dict = torch.load(resume_model)
  model.load_state_dict(model_state_dict)
else:
    logging.info("WARNING: You didn't provide a path to resume the model (--resume_model parameter). " +
                 "Evaluation will be computed using randomly initialized weights.")

model = model.to(args.device)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
DEBUG:root:Train only layer3 and layer4 of the resnet18, freeze the previous ones
INFO:root:There are 1 GPUs and 2 CPUs.
INFO:root:Loading model from /content/logs/default/2022-12-13_12-16-57/best_model.pth


## Test on SF-XS

Test the model on the sf-xs (test) dataset.

In [ ]:
# dataset_folder is the same of the training
test_set_folder = os.path.join(DATASET_FOLDER, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder, queries_folder="queries_v1",
                      positive_dist_threshold=args.positive_dist_threshold)

recalls, recalls_str = test.test(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
sf_xs_r15 = f"{recalls[0]}/{recalls[1]}"

DEBUG:root:Extracting database descriptors for evaluation/testing
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|███████████████████████████████████████████████████████████| 1700/1700 [03:41<00:00,  7.69it/s]
DEBUG:root:Extracting queries descriptors for evaluation/testing using batch size 1
100%|███████████████████████████████████████████████████████████| 1000/1000 [00:16<00:00, 61.62it/s]
DEBUG:root:Calculating recalls
INFO:root:< test - #q: 1000; #db: 27191 >: R@1: 52.2, R@5: 66.3, R@10: 71.8, R@20: 76.3


## Test on Tokyo-XS

Test the model on the tokyo-xs dataset.

In [ ]:
tokyo_xs_folder = "/content/tokyo_xs"
test_set_folder = os.path.join(tokyo_xs_folder, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder,
                      positive_dist_threshold=args.positive_dist_threshold)

recalls, recalls_str = test.test(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
tokyo_xs_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

DEBUG:root:Extracting database descriptors for evaluation/testing
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|█████████████████████████████████████████████████████████████| 799/799 [02:09<00:00,  6.18it/s]
DEBUG:root:Extracting queries descriptors for evaluation/testing using batch size 1
100%|█████████████████████████████████████████████████████████████| 315/315 [00:05<00:00, 55.87it/s]
DEBUG:root:Calculating recalls
INFO:root:< test - #q: 315; #db: 12771 >: R@1: 69.5, R@5: 84.8, R@10: 89.2, R@20: 93.0


## Test on Tokyo-Night

Test the model on the tokyo-night dataset.

In [ ]:
tokyo_night_folder = "/content/tokyo-night/"
test_set_folder = os.path.join(tokyo_night_folder, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder,
                      positive_dist_threshold=args.positive_dist_threshold)

recalls, recalls_str = test.test(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
tokyo_night_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

DEBUG:root:Extracting database descriptors for evaluation/testing
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|█████████████████████████████████████████████████████████████| 799/799 [02:06<00:00,  6.29it/s]
DEBUG:root:Extracting queries descriptors for evaluation/testing using batch size 1
100%|█████████████████████████████████████████████████████████████| 105/105 [00:02<00:00, 41.85it/s]
DEBUG:root:Calculating recalls
INFO:root:< test - #q: 105; #db: 12771 >: R@1: 50.5, R@5: 72.4, R@10: 79.0, R@20: 85.7


# Save results

## Create CSV with recalls

In [ ]:
import csv
header = ["sf-xs (test)", "Tokyo-xs", "Tokyo-night"]
data = [sf_xs_r15, tokyo_xs_r15, tokyo_night_r15]

logging.info(f"Table results: {zip(header, data)}")

with open(f"/content/{SAVEDIR}_{start_time.strftime('%Y-%m-%d_%H-%M-%S')}.csv", "w") as f:
  writer = csv.writer(f)

  writer.writerow(header)
  writer.writerow(data)
logging.info(f"save table results to /content/{SAVEDIR}{start_time.strftime('%Y-%m-%d_%H-%M-%S')}.csv")

## Save on Drive

#### ArcFace

In [ ]:
modelLocal = "arcface_model.pth"

#### SphereFace

In [ ]:
modelLocal = "sphereface_model.pth"

#### CosFace

In [ ]:
modelLocal = "cosface_model.pth"

Save all data generated by this notebook in a specific folder in **PERSONAL** gdrive. Remember to copy inside shared_data of project drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

if not os.path.exists("/content/drive/MyDrive/step_3"):
  !mkdir /content/drive/MyDrive/step_3
  !cp f"/content/logs/{SAVEDIR}/{start_time.strftime('%Y-%m-%d_%H-%M-%S')}/best_model.pth" f"/content/drive/MyDrive/step_3/{modelLocal}"
  

#!cp "/content/{SAVEDIR}{start_time.strftime('%Y-%m-%d_%H-%M-%S')}.csv" /content/drive/MyDrive/step_2/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
updating: content/logs/ (stored 0%)
updating: content/logs/content/ (stored 0%)
updating: content/logs/content/saved_models/ (stored 0%)
updating: content/logs/content/saved_models/2022-12-11_17-17-27/ (stored 0%)
updating: content/logs/content/saved_models/2022-12-11_17-17-27/debug.log (deflated 66%)
updating: content/logs/content/saved_models/2022-12-11_17-17-27/info.log (deflated 58%)
updating: content/logs/content/saved_models/2022-12-11_17-17-27/best_model.pth (deflated 7%)
updating: content/logs/content/saved_models/2022-12-11_17-17-27/last_checkpoint.pth (deflated 8%)
updating: content/logs/default/ (stored 0%)
updating: content/logs/default/2022-12-13_12-16-57/ (stored 0%)
updating: content/logs/default/2022-12-13_12-16-57/debug.log (deflated 67%)
updating: content/logs/default/2022-12-13_12-16-57/info.log (deflated 58%)
updating: content/logs/default